# Baseline Experiment - Santa 2025

Copy the best available submission from snapshots and calculate its score.

In [ ]:
import numpy as np
import pandas as pd
import math
from numba import njit
import os
import shutil

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125], dtype=np.float64)
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5], dtype=np.float64)

In [ ]:
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a group of trees."""
    n = xs.size
    V = tx.size
    mnx = 1e300
    mny = 1e300
    mxx = -1e300
    mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx:
                mnx = X
            if X > mxx:
                mxx = X
            if Y < mny:
                mny = Y
            if Y > mxy:
                mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    """Remove 's' prefix from values."""
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def calculate_total_score(df, tx, ty):
    """Calculate total score for a submission."""
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    total_score = 0.0
    scores_by_n = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        sc = score_group(xs, ys, ds, tx, ty)
        scores_by_n[n] = sc
        total_score += sc
    return total_score, scores_by_n

In [ ]:
# Load the best snapshot submission
best_snapshot_path = '/home/nonroot/snapshots/santa-2025/21322576451/submission/submission.csv'
df = pd.read_csv(best_snapshot_path)
print(f"Loaded submission with {len(df)} rows")
print(df.head())

In [ ]:
# Calculate the score
total_score, scores_by_n = calculate_total_score(df, TX, TY)
print(f"\nTotal Score: {total_score:.6f}")
print(f"\nScore breakdown by N range:")
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores_by_n[n] for n in range(start, end+1))
    pct = range_score / total_score * 100
    print(f"  N={start}-{end}: {range_score:.4f} ({pct:.1f}%)")

In [ ]:
# Copy to submission folder
shutil.copy(best_snapshot_path, '/home/submission/submission.csv')
print("Copied to /home/submission/submission.csv")

# Also save to experiment folder
shutil.copy(best_snapshot_path, '/home/code/experiments/001_baseline/submission.csv')
print("Copied to experiment folder")

In [ ]:
# Save metrics
import json
metrics = {'cv_score': total_score}
with open('/home/code/experiments/001_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Saved metrics: {metrics}")

In [ ]:
# Verify submission format
print("\nSubmission format check:")
print(f"Columns: {list(df.columns)}")
print(f"Total rows: {len(df)}")
print(f"Expected rows: {sum(range(1, 201))} (sum of 1 to 200)")
print(f"\nFirst few rows:")
print(df.head(10))
print(f"\nLast few rows:")
print(df.tail(5))